# Neural-based classification rule learning for sequential data

## Install and Import used packages

In [8]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from src.run import train, test, test_rule

In [10]:
path_checkpoint = "peptides_checkpoint.pt"

## Parameters

### Data

In [11]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile
import os

if not os.path.exists("ACPs_Breast_cancer.csv"):
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00589/Anticancer_Peptides.zip'
    r = requests.get(url)
    z = ZipFile(BytesIO(r.content))
    df = pd.read_csv(z.open("epitope.csv"))
    df.to_csv("epitope.csv")
else:
    df = pd.read_csv("epitope.csv")
df

Unnamed: 0    ID                               sequence  class
0              1     1          0 1 2 3 4 5 1 6 1 5 3 5 7 8 3      0
1              2     2   2 9 10 0 7 11 12 13 13 5 11 11 7 7 7      0
2              3     3  8 11 0 10 2 14 15 16 2 17 3 0 14 9 10      0
3              4     4  16 17 17 7 0 14 9 5 16 11 5 3 16 4 14      0
4              5     5  14 11 7 7 0 10 5 9 10 17 7 16 10 9 14      0
...          ...   ...                                    ...    ...
2387        2388  2388   16 1 4 7 11 4 1 5 17 4 16 16 1 11 11      1
2388        2389  2389  11 1 5 4 17 19 8 16 19 1 4 12 2 3 3 0      1
2389        2390  2390    4 4 4 0 4 18 7 7 2 17 19 4 10 16 16      1
2390        2391  2391   5 15 2 16 4 0 1 8 14 3 11 13 14 3 12      1
2391        2392  2392   1 17 17 19 0 16 16 8 2 0 1 5 7 16 16      1

[2392 rows x 4 columns]

In [13]:
batch_size=100
val_size=0.2
test_size=0.2

In [14]:
from src.data.datasets import UCIAnticancerDataset
from src.data.util import construct_data

dataset = UCIAnticancerDataset(df)
train_loader, val_loader, test_loader = construct_data(dataset, batch_size=batch_size, val_size=val_size, test_size=test_size, seed=42)

Distrib 0.5585284280936454 1336.0 2392


c:\Users\14153\Downloads\cr2n-main\cr2n-main\src\data\util.py:18: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  sampler = BucketBatchSampler(np.array(dataset.x, dtype=object)[indices], batch_size)


### Model

In [15]:
input_size = len(dataset.features_name)
window_size = 3
base_model_hidden_size = window_size * 2
base_or_output_size=1
output_size=1

pad_border = True

max_sequence_length = max(
    train_loader.batch_sampler.max_length,
    val_loader.batch_sampler.max_length,
    test_loader.batch_sampler.max_length
)

# Compute dimension of convOR input size
conv_dim_out = max_sequence_length - (window_size - 1)
if pad_border:
    conv_dim_out = conv_dim_out + (window_size - 1) * 2

### Training

In [16]:
from src.sparsify import Pruning

pruning = Pruning()
pruning30 = Pruning(start=30)
no_pruning = False

In [17]:
pruning_strategy = pruning # or no_pruning or pruning30
epochs = 200
# Other parameters are fixed directly in train() function

## Run Local model

In [18]:
from src.models import LocalModel

for i in range(10):
    local_model = LocalModel(input_size, window_size, pad_border, max_sequence_length, base_model_hidden_size, conv_dim_out, base_or_output_size, output_size)
    train(local_model, train_loader, val_loader, pruning=pruning_strategy, epochs=epochs, path_checkpoint=path_checkpoint, verbose=True)
    rule = local_model.extract_rule(features_names=dataset.features_name, verbose=False)
    test(local_model.model, test_loader)
    test_rule(rule, test_loader)
    print('-----')

In [ ]:
print(rule)

(((A at t-73 OR D at t-73 OR E at t-73 OR F at t-73 OR G at t-73 OR I at t-73 OR L at t-73 OR M at t-73 OR P at t-73 OR Q at t-73 OR S at t-73 OR T at t-73 OR V at t-73 OR W at t-73 OR Y at t-73) AND (A at t-72 OR C at t-72 OR D at t-72 OR E at t-72 OR F at t-72 OR G at t-72 OR H at t-72 OR I at t-72 OR L at t-72 OR M at t-72 OR N at t-72 OR Q at t-72 OR S at t-72 OR T at t-72 OR V at t-72 OR W at t-72 OR Y at t-72) AND (A at t-71 OR C at t-71 OR D at t-71 OR E at t-71 OR F at t-71 OR G at t-71 OR H at t-71 OR I at t-71 OR L at t-71 OR M at t-71 OR N at t-71 OR P at t-71 OR Q at t-71 OR S at t-71 OR T at t-71 OR V at t-71 OR W at t-71 OR Y at t-71))
OR
((A at t-4 OR D at t-4 OR E at t-4 OR F at t-4 OR G at t-4 OR I at t-4 OR L at t-4 OR M at t-4 OR P at t-4 OR Q at t-4 OR S at t-4 OR T at t-4 OR V at t-4 OR W at t-4 OR Y at t-4) AND (A at t-3 OR C at t-3 OR D at t-3 OR E at t-3 OR F at t-3 OR G at t-3 OR H at t-3 OR I at t-3 OR L at t-3 OR M at t-3 OR N at t-3 OR Q at t-3 OR S at t-3 O

## Run Global Model

In [ ]:
from src.models import GlobalModel

for i in range(10):
    global_model = GlobalModel(input_size, window_size, pad_border, max_sequence_length, base_model_hidden_size, conv_dim_out, base_or_output_size, output_size)
    train(global_model, train_loader, val_loader, pruning=pruning_strategy, epochs=epochs, path_checkpoint=path_checkpoint, verbose=True)
    rule = global_model.extract_rule(features_names=dataset.features_name, verbose=False)
    test(global_model.model, test_loader)
    test_rule(rule, test_loader)
    print('-----')

[200] train loss: 0.453523027, val loss: 0.500000000 | best val acc: 0.56, best penalty: 38.91
=> loaded checkpoint 'peptides_checkpoint.pt' (epoch 15, acc 0.5566111111111111, loss 0.4433888792991638, penalty 38.90810775756836)
network test accuracy: 55.6%, balanced: 55.6% (TP:7529, FN:1471, FP:6513, TN:2487)
rule test accuracy: 55.6%, balanced: 55.6% (TP:7529, FN:1471, FP:6513, TN:2487)
  Expr 0     - triggered 14042 - acc: 53.6% (TP:7529, FN:0, FP:6513, TN:0)
  Expr None  - triggered 3958 - acc: 62.8% (TP:0, FN:1471, FP:0, TN:2487)
-----
[200] train loss: 0.449162355, val loss: 0.483111113 | best val acc: 0.56, best penalty: 37.09
=> loaded checkpoint 'peptides_checkpoint.pt' (epoch 5, acc 0.5561666666666667, loss 0.4438333213329315, penalty 37.08836364746094)
network test accuracy: 55.6%, balanced: 55.6% (TP:5681, FN:3319, FP:4669, TN:4331)
rule test accuracy: 55.6%, balanced: 55.6% (TP:5681, FN:3319, FP:4669, TN:4331)
  Expr 0     - triggered 10350 - acc: 54.9% (TP:5681, FN:0, FP:4

KeyboardInterrupt: 

In [ ]:
rule = global_model.extract_rule(features_names=dataset.features_name, verbose=False)
test(global_model.model, test_loader)
test_rule(rule, test_loader)
print('-----')
print(rule)

network test accuracy: 54.7%, balanced: 54.7% (TP:6242, FN:2758, FP:5402, TN:3598)
rule test accuracy: 54.7%, balanced: 54.7% (TP:6242, FN:2758, FP:5402, TN:3598)
  Expr 0     - triggered 11644 - acc: 53.6% (TP:6242, FN:0, FP:5402, TN:0)
  Expr None  - triggered 6356 - acc: 56.6% (TP:0, FN:2758, FP:0, TN:3598)
-----
(D OR E OR F OR I OR M OR N OR Q OR T OR V OR Y)-(C OR I OR L OR M OR S OR V OR W OR Y)-(A OR D OR F OR I OR M OR N OR Q OR R OR V OR Y) in sequence
